In [8]:
import json
import polars as pl
import os

In [41]:
df = pl.read_parquet("./clean-data/hotelesv2.parquet")

In [42]:
df=df.drop(['name',
 'descrption',
 'latitude',
 'longitude',
 'category',
 'avg_rating',
 'num_of_reviews',
 'County',
 'City',
 'State',
 'Country'])

In [9]:
def contar_archivos_por_carpeta(ruta):
    archivos_por_carpeta = {}
    
    for nombre in os.listdir(ruta):
        ruta_completa = os.path.join(ruta, nombre)
        if os.path.isdir(ruta_completa):
            archivos_por_carpeta[nombre] = len(os.listdir(ruta_completa))

    return archivos_por_carpeta

In [10]:
carpetas = contar_archivos_por_carpeta('./reviews/reviews-estados')

In [37]:
def transform (carpeta,archivo):
    with open(f'./reviews/reviews-estados/{carpeta}/{archivo}.json', 'r') as f:
        texto = f.read()

    lineas = texto.split('\n')
    user_ids, names, times, ratings, texts, gmap_ids = [], [], [], [], [], []
    
    for linea in lineas:
        if linea.strip():  # Ignora líneas en blanco
            objeto_json = json.loads(linea)
            user_ids.append(objeto_json['user_id'])
            names.append(objeto_json['name'])
            times.append(objeto_json['time'])
            ratings.append(objeto_json['rating'])
            texts.append(objeto_json['text'])
            gmap_ids.append(objeto_json['gmap_id'])
            
    data = {
    'user_id':user_ids,
    'name':names,
    'time':times,
    'rating':ratings,
    'text':texts,
    'gmap_id':gmap_ids
            }
    
    df = pl.DataFrame(data)
    df = df.filter(~pl.col('gmap_id').is_null())
    df = df.unique(subset=['gmap_id'], keep='first')
    return df

In [43]:
for carpeta,cant in carpetas.items():
    dfa = transform(carpeta,1)
    dfa = dfa.join(df, on='gmap_id', how='inner')
    for i in range(2,cant+1):
        dfb = transform(carpeta,i)
        dfb = dfb.join(df, on='gmap_id', how='inner')
        dfa = dfa.extend(dfb)
    dfa.write_parquet(f'./reviews/reviews-data/{carpeta}.parquet')

In [49]:
a1 = list(carpetas.keys())[0]
print(type(a1), a1)

<class 'str'> review-Alabama


In [51]:
a1 = list(carpetas.keys())[0]
df1 = pl.read_parquet(f'./reviews/reviews-data/{a1}.parquet')
for carpeta, cant in carpetas.items():
    if carpeta == a1:
        continue
    df2 = pl.read_parquet(f'./reviews/reviews-data/{carpeta}.parquet')
    df1 = df1.extend(df2)
df1.write_parquet('./reviews/reviews-data/all-reviews.parquet')

In [52]:
df1

user_id,name,time,rating,text,gmap_id,facilities,url
str,str,i64,i64,str,str,list[str],str
"""11262285274252…","""kitt Tidwell""",1615917077406,2,"""I lifted the b…","""0x88626b3eef63…",null,"""https://www.go…"
"""10470722570413…","""Dottie Engle""",1535302758918,5,"""Very polite fr…","""0x889a8a0f1624…",null,"""https://www.go…"
"""11361327080121…","""Ashley Peters""",1494551764821,5,"""Amazing hotel …","""0x889a04ce45e4…",null,"""https://www.go…"
"""11499210839610…","""Cherena Dukes""",1574336225144,4,"""Nice homes in …","""0x88892dc32d67…",null,"""https://www.go…"
"""11266518652829…","""Charlie Cline""",1454821094934,4,"""Good location.…","""0x88890491e941…",null,"""https://www.go…"
"""10935668939329…","""Jennifer Gurle…",1515726182259,4,"""The rooms are …","""0x888bb13bb3be…",null,"""https://www.go…"
"""11645382291336…","""Curtis Mitchel…",1540831167875,3,"""Ever been in t…","""0x88626b5f2310…",null,"""https://www.go…"
"""10960068563625…","""Miranda Morris…",1610719146150,5,"""Its no longer …","""0x888bd12a241f…",null,"""https://www.go…"
"""11491755808860…","""Robert Nilsson…",1546210453899,5,"""The best place…","""0x889a05c8ea6e…",null,"""https://www.go…"
